In [2]:
import SimpleITK as sitk
import os
import numpy as np
import shutil
import random

In [3]:
classes = {
    "aorta.nii.gz": 1,
    "heart_atrium_left.nii.gz": 2,
    "heart_atrium_right.nii.gz": 3,
    "heart_myocardium.nii.gz": 4,
    "heart_ventricle_left.nii.gz": 5,
    "heart_ventricle_right.nii.gz": 6,
}

In [4]:
reader = sitk.ImageFileReader()
reader.SetImageIO("NiftiImageIO")
writer = sitk.ImageFileWriter()
writer.SetFileName("cropped_mask.nii.gz")
images_folder = "imagesTr"
labels_folder = "labelsTr"

shutil.rmtree(images_folder, ignore_errors=True)
shutil.rmtree(labels_folder, ignore_errors=True)
os.mkdir(images_folder)
os.mkdir(labels_folder)
sorted_scans = sorted(os.listdir("data"))
data_folder = os.path.join(os.environ["HOME"], "data")
cnt = 0
hide_one_class_at_random = False
for scan_id in sorted_scans[255:]:
    reader.SetFileName(os.path.join(data_folder, scan_id, "ct.nii.gz"))
    try:
        full_scan = sitk.GetArrayFromImage(reader.Execute())
    except:
        print("bad scan", scan_id)
        continue
    total_mask = np.zeros(full_scan.shape)
    for key in classes:
        segment_filename = os.path.join(
            data_folder, scan_id, "segmentations", key
        )
        if not os.path.exists(segment_filename):
            print("incomplete segmentation", scan_id)
            continue
        reader.SetFileName(segment_filename)
        total_mask += classes[key] * sitk.GetArrayFromImage(reader.Execute())
    indices = np.where(total_mask > 0)
    if indices[0].size > 0:
        min_max = [(i.min(), i.max()) for i in indices]
        cropped_mask = total_mask[
            min_max[0][0] : min_max[0][1] + 1,
            min_max[1][0] : min_max[1][1] + 1,
            min_max[2][0] : min_max[2][1] + 1,
        ]
        cropped_scan = full_scan[
            min_max[0][0] : min_max[0][1] + 1,
            min_max[1][0] : min_max[1][1] + 1,
            min_max[2][0] : min_max[2][1] + 1,
        ]
        if hide_one_class_at_random:
            hidden_class = random.choice(range(1, 7))
            cropped_mask[cropped_mask == hidden_class] = 0
        writer.SetFileName(
            os.path.join(labels_folder, f"la_{scan_id[1:]}.nii.gz")
        )
        writer.Execute(sitk.GetImageFromArray(cropped_mask))
        writer.SetFileName(
            os.path.join(images_folder, f"la_{scan_id[1:]}_0000.nii.gz")
        )
        writer.Execute(sitk.GetImageFromArray(cropped_scan))
        cnt += 1
        if cnt >= 100:
            break

bad scan s0326
bad scan s0352
bad scan s0353
bad scan s0361
bad scan s0386
bad scan s0417
bad scan s0436


In [5]:
import os
import SimpleITK as sitk
from tqdm import tqdm

reader = sitk.ImageFileReader()
reader.SetImageIO("NiftiImageIO")

for filename in tqdm(os.listdir("labelsTr")):
    reader.SetFileName(os.path.join("labelsTr", filename))
    scan = reader.Execute()
    values = [i in sitk.GetArrayFromImage(scan) for i in range(7)]
    if 6 > sum(values):
        print(filename, values)

  4%|▍         | 4/100 [00:00<00:03, 26.21it/s]

la_0316.nii.gz [True, True, False, False, False, False, False]


  7%|▋         | 7/100 [00:00<00:03, 27.17it/s]

la_0442.nii.gz [True, True, False, False, False, False, False]


 13%|█▎        | 13/100 [00:00<00:04, 19.73it/s]

...

la_0359.nii.gz [True, True, False, False, False, False, False]


 83%|████████▎ | 83/100 [00:03<00:00, 23.21it/s]

la_0379.nii.gz [True, True, False, False, False, False, False]


100%|██████████| 100/100 [00:05<00:00, 19.57it/s]

la_0400.nii.gz [True, True, False, False, False, False, False]
